##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 指標とオプティマイザを移行する

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/metrics_optimizers"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/metrics_optimizers.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/metrics_optimizers.ipynb"> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/migrate/metrics_optimizers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

TF1 では、`tf.metrics` はすべての指標関数の API 名前空間です。各指標は、`label` と `prediction` を入力パラメータとして取り、対応する指標テンソルを結果として返す関数です。TF2 では、`tf.keras.metrics` にすべての指標関数とオブジェクトが含まれています。`Metric` オブジェクトを `tf.keras.Model` および `tf.keras.layers.layer` で使用して、指標値を計算できます。

## セットアップ

いくつかの必要な TensorFlow インポートから始めましょう。

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

デモ用にいくつかの簡単なデータを準備します。

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [0, 0, 1]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [0, 1, 1]

## TF1: Estimator を使用した tf.compat.v1.metrics

TF1 では、指標は `eval_metric_ops` として `EstimatorSpec` に追加でき、演算は　`tf.metrics`　で定義されたすべての指標関数を介して生成されます。例に従って、`tf.metrics.accuracy` の使用方法を確認できます。

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(2)(features)
  predictions = tf.math.argmax(input=logits, axis=1)
  loss = tf1.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  accuracy = tf1.metrics.accuracy(labels=labels, predictions=predictions)
  return tf1.estimator.EstimatorSpec(mode, 
                                     predictions=predictions,
                                     loss=loss, 
                                     train_op=train_op,
                                     eval_metric_ops={'accuracy': accuracy})

estimator = tf1.estimator.Estimator(model_fn=_model_fn)
estimator.train(_input_fn)

In [ ]:
estimator.evaluate(_eval_input_fn)

また、指標は `tf.estimator.add_metrics()` を介してエスティメータに直接追加できます。

In [ ]:
def mean_squared_error(labels, predictions):
  labels = tf.cast(labels, predictions.dtype)
  return {"mean_squared_error": 
          tf1.metrics.mean_squared_error(labels=labels, predictions=predictions)}

estimator = tf1.estimator.add_metrics(estimator, mean_squared_error)
estimator.evaluate(_eval_input_fn)

## TF2: tf.keras.Model を使用した Keras メトリクス API

TF2 では、`tf.keras.metrics` にすべての指標クラスと関数が含まれています。これらは OOP スタイルで設計されており、他の `tf.keras` API と密接に統合されています。すべての指標は `tf.keras.metrics` 名前空間で見つけることができ、通常は `tf.compat.v1.metrics` と `tf.keras.metrics` の間に直接マッピングがあります。

次の例では、指標が `model.compile()` メソッドに追加されています。ユーザーは、ラベルと予測テンソルを指定せずに、指標インスタンスを作成するだけで済みます。Keras モデルは、モデルの出力とラベルを指標オブジェクトにルーティングします。

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

inputs = tf.keras.Input((2,))
logits = tf.keras.layers.Dense(2)(inputs)
predictions = tf.math.argmax(input=logits, axis=1)
model = tf.keras.models.Model(inputs, predictions)
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer, loss='mse', metrics=[tf.keras.metrics.Accuracy()])

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

Eager execution を有効にすると、`tf.keras.metrics.Metric` インスタンスを直接使用して、numpy データまたは Eager テンソルを評価できます。`tf.keras.metrics.Metric` オブジェクトはステートフルコンテナーです。指標値は `metric.update_state(y_true, y_pred)` で更新でき、結果は `metrics.result() `で取得できます。


In [ ]:
accuracy = tf.keras.metrics.Accuracy()

accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[0, 0, 0, 1])
accuracy.result().numpy()


In [ ]:
accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[0, 0, 0, 0])
accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[1, 1, 0, 0])
accuracy.result().numpy()

`tf.keras.metrics.Metric` の詳細については、`tf.keras.metrics.Metric` の API ドキュメントと[移行ガイド](https://www.tensorflow.org/guide/effective_tf2#new-style_metrics_and_losses)を参照してください。

## TF1.x オプティマイザの Keras オプティマイザへの移行

<a>Adam オプティマイザ</a>や[勾配降下オプティマイザ](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/AdamOptimizer)などの <code>tf.compat.v1.train</code> 内のオプティマイザは、`tf.keras.optimizers` 内に同等のものをもちます。

以下の表は、これらのレガシーオプティマイザを Keras の同等のものに変換する方法をまとめたものです。追加の手順（[デフォルトの学習率の更新](../../guide/effective_tf2.ipynb#optimizer_defaults)など）が必要でない限り、TF1.x バージョンを TF2 バージョンに直接置き換えることができます。

オプティマイザを変換すると、[古いチェックポイントの互換性が失われる可能性があること](./migrating_checkpoints.ipynb)に注意してください。

<table>
  <tr>
    <th>TF1.x</th>
    <th>TF2</th>
    <th>追加の手順</th>
  </tr>
  <tr>
    <td>`tf.v1.train.GradientDescentOptimizer`</td>
    <td>`tf.keras.optimizers.SGD`</td>
    <td>なし</td>
  </tr>
  <tr>
    <td>`tf.v1.train.MomentumOptimizer`</td>
    <td>`tf.keras.optimizers.SGD`</td>
    <td>`momentum` 引数を含む</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdamOptimizer`</td>
    <td>`tf.keras.optimizers.Adam`</td>
    <td>`beta1` および `beta2` 引数の名前を `beta_1` および `beta_2` に変更する</td>
  </tr>
  <tr>
    <td>`tf.v1.train.RMSPropOptimizer`</td>
    <td>`tf.keras.optimizers.RMSprop`</td>
    <td>`decay` 引数の名前を `rho` に変更する</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdadeltaOptimizer`</td>
    <td>`tf.keras.optimizers.Adadelta`</td>
    <td>なし</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdagradOptimizer`</td>
    <td>`tf.keras.optimizers.Adagrad`</td>
    <td>なし</td>
  </tr>
  <tr>
    <td>`tf.v1.train.FtrlOptimizer`</td>
    <td>`tf.keras.optimizers.Ftrl`</td>
    <td>`accum_name` および `linear_name` 引数を削除する</td>
  </tr>
  <tr>
    <td>`tf.contrib.AdamaxOptimizer`</td>
    <td>`tf.keras.optimizers.Adamax`</td>
    <td>`beta1` および `beta2` 引数の名前を `beta_1` および `beta_2` に変更する</td>
  </tr>
  <tr>
    <td>`tf.contrib.Nadam`</td>
    <td>`tf.keras.optimizers.Nadam`</td>
    <td>`beta1` および `beta2` 引数の名前を `beta_1` および `beta_2` に変更する</td>
  </tr>
</table>

注意: TF2 では、すべてのイプシロン（数値安定定数）のデフォルトが `1e-8` ではなく `1e-7` になりました。ほとんどの場合、この違いは無視できます。